In [1]:
%pip install selenium
from selenium import webdriver


driver = webdriver.Chrome()
driver.get('https://selenium.dev/')
driver.quit()


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached selenium-4.32.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (9

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
import os # For screenshots

def scrape_xueqiu_post_and_comments(url):
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless") # Keep headless commented out for debugging popups
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36") # Add a common user agent
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    # Create a directory for screenshots if it doesn't exist
    if not os.path.exists("screenshots"):
        os.makedirs("screenshots")

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()

    scraped_data = {"main_post": None, "comments": []}
    wait = WebDriverWait(driver, 15) # Adjust wait time if needed (15s is usually enough)

    try:
        print(f"Navigating to: {url}")
        driver.get(url)

        # --- Handle First Pop-up (Login - "跳过") ---
        try:
            print("Looking for the first pop-up ('跳过')...")
            skip_button_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"
            skip_button = wait.until(EC.element_to_be_clickable((By.XPATH, skip_button_xpath)))
            print("First pop-up '跳过' button found. Attempting to click...")
            driver.execute_script("arguments[0].click();", skip_button)
            print("Clicked '跳过'.")
            time.sleep(1) # Short pause after click
        except TimeoutException:
            print("First pop-up ('跳过') did not appear or was not found within timeout.")
            driver.save_screenshot("screenshots/debug_no_first_popup.png")
        except Exception as e:
            print(f"Error handling first pop-up: {e}")
            driver.save_screenshot("screenshots/debug_error_first_popup.png")

        # --- Add Pause for Manual Inspection ---
        # print(">>> Pausing for 45 seconds for manual inspection of potential second pop-up... <<<")
        # print(">>> Please inspect the 'X' button and check for iframes. Copy its XPath/Selector. <<<")
        # time.sleep(45)
        # print(">>> Resuming script... <<<")
        # --- OR Take screenshot for headless debugging ---
        print("Taking screenshot before handling second popup...")
        driver.save_screenshot("screenshots/debug_before_second_popup.png")
        print("Screenshot saved to screenshots/debug_before_second_popup.png")


        # --- Handle Second Pop-up (Close "X") ---
        close_button_found = False
        try:
            print("Looking for the second pop-up (close button 'X')...")

            # === OPTION A: Check for iframe first (if suspected) ===
            iframe_xpath = "//iframe[contains(@id, 'dialog') or contains(@class, 'modal-iframe')]" # Example iframe locator - ADJUST THIS
            try:
                print("Checking for known iframe patterns...")
                iframe = wait.until(EC.presence_of_element_located((By.XPATH, iframe_xpath)))
                print("Potential iframe found. Switching to iframe...")
                driver.switch_to.frame(iframe)
                print("Successfully switched to iframe. Now looking for close button inside iframe...")

                # --- Find 'X' button INSIDE iframe ---
                # Adjust this XPath based on inspection *inside* the iframe
                iframe_close_button_xpath = "//i[contains(@class, 'close')] | //button[contains(@aria-label, 'Close') or contains(text(), '关闭')]"
                close_button = wait.until(EC.element_to_be_clickable((By.XPATH, iframe_close_button_xpath)))
                print(f"Second pop-up 'X' button found INSIDE IFRAME. Clicking...")
                driver.execute_script("arguments[0].click();", close_button)
                print("Clicked 'X' inside iframe.")
                close_button_found = True
                time.sleep(1)
                print("Switching back to default content.")
                driver.switch_to.default_content() # IMPORTANT: Switch back

            except TimeoutException:
                print("No iframe found matching pattern, or 'X' button not found inside iframe. Proceeding to check main page.")
                # If iframe was found but button wasn't, need to switch back
                try:
                    driver.switch_to.default_content()
                except: pass # Already in default if iframe wasn't found

            except Exception as e_iframe:
                 print(f"Error during iframe handling: {e_iframe}")
                 try:
                     driver.switch_to.default_content() # Ensure switch back on error
                 except: pass


            # === OPTION B: Look for 'X' on the main page (if not found in iframe or no iframe suspected) ===
            if not close_button_found:
                print("Looking for close button on the main page...")
                # ADD THE XPATH YOU FOUND DURING MANUAL INSPECTION HERE!
                # Example: close_button_xpaths = ["//button[@aria-label='Close Dialog']", ...]
                close_button_xpaths = [
                    # Add your best guess from inspection first:
                    "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", # Common Xueqiu pattern
                    "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", # Another dialog pattern
                    "//i[contains(@class, 'cube-dialog-close')]", # Seen on some Xueqiu popups
                    "//div[contains(@class, 'modal-container')]//i[contains(@class, 'close')]",
                    "//div[contains(@class, 'dialog')]//i[contains(@class, 'close')]",
                    "//button[@aria-label='Close']", # Common accessibility pattern
                    "//button[contains(@class, 'close')]"
                ]
                for xpath in close_button_xpaths:
                    try:
                        print(f"Trying XPath: {xpath}")
                        close_button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
                        print(f"Second pop-up 'X' button found with XPath: {xpath}. Clicking...")
                        driver.execute_script("arguments[0].click();", close_button)
                        print("Clicked 'X' on the second pop-up (main page).")
                        close_button_found = True
                        time.sleep(1) # Pause after click
                        break # Exit loop once found and clicked
                    except TimeoutException:
                        print(f"Close button with XPath '{xpath}' not found or not clickable yet.")
                        continue # Try next XPath
                    except ElementClickInterceptedException:
                         print(f"Close button found ({xpath}) but click was intercepted. Trying JS click again forcefully.")
                         try:
                             driver.execute_script("arguments[0].click();", close_button)
                             print("Clicked 'X' via JS after interception.")
                             close_button_found = True
                             time.sleep(1)
                             break
                         except Exception as e_intercept:
                             print(f"Force JS click also failed: {e_intercept}")
                             driver.save_screenshot(f"screenshots/debug_click_intercepted_{xpath.replace('/','_').replace('@','')}.png")
                             continue # Try next XPath
                    except Exception as e_click:
                         print(f"Error clicking button with XPath '{xpath}': {e_click}")
                         driver.save_screenshot(f"screenshots/debug_click_error_{xpath.replace('/','_').replace('@','')}.png")
                         continue # Try next XPath


                if not close_button_found:
                     print("Second pop-up (close 'X') did not appear or was not found with any attempted XPaths on main page.")
                     driver.save_screenshot("screenshots/debug_no_second_popup_found.png")


        except Exception as e:
            print(f"An error occurred during the second pop-up handling phase: {e}")
            driver.save_screenshot("screenshots/debug_error_second_popup_phase.png")


        # --- Scrape Main Post Content ---
        # (Wait longer if popups might have delayed page load)
        content_wait = WebDriverWait(driver, 20)
        try:
            print("Scraping main post content...")
            post_content_element = content_wait.until(EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'article__content') or contains(@class, 'article__bd__detail')]")))
            scraped_data["main_post"] = post_content_element.text
            print("Main post content scraped.")
        except TimeoutException:
            print("Main post content not found.")
            driver.save_screenshot("screenshots/debug_no_main_post.png")
        except Exception as e:
            print(f"Error scraping main post: {e}")
            driver.save_screenshot("screenshots/debug_error_main_post.png")

        # --- Click on Comments Tab ---
        try:
            print("Looking for and clicking the '评论' (Comments) tab...")
            comments_tab_xpath = "//div[contains(@class, 'tab__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')] | //div[text()='评论' and contains(@class,'tab__control')]"
            comments_tab = wait.until(EC.element_to_be_clickable((By.XPATH, comments_tab_xpath)))
            print("Comments tab found. Clicking...")
            # Scroll into view before clicking, sometimes helps
            driver.execute_script("arguments[0].scrollIntoView(true);", comments_tab)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", comments_tab)
            print("Clicked '评论' tab.")
            time.sleep(3) # Wait for comments section to potentially load/animate
        except TimeoutException:
            print("Comments tab not found or not clickable.")
            driver.save_screenshot("screenshots/debug_no_comment_tab.png")
        except Exception as e:
            print(f"Error clicking comments tab: {e}")
            driver.save_screenshot("screenshots/debug_error_comment_tab.png")


        # --- Scrape Comments ---
        try:
            print("Scraping comments...")
            # Wait for the container of comments to be present
            comments_container_xpath = "//div[contains(@class, 'reply-list')]"
            wait.until(EC.presence_of_element_located((By.XPATH, comments_container_xpath)))
            print("Comments container found.")

            # Wait for at least one comment text element to be visible
            comment_text_xpath = "//div[contains(@class, 'reply-item__main')]//div[contains(@class, 'reply-item__content')]"
            wait.until(EC.visibility_of_element_located((By.XPATH, f"({comment_text_xpath})[1]"))) # Wait for the first one
            print("First comment text element visible.")

            # Find all comment text elements
            comment_elements = driver.find_elements(By.XPATH, comment_text_xpath)

            if comment_elements:
                print(f"Found {len(comment_elements)} potential comment elements.")
                for i, comment_el in enumerate(comment_elements):
                    try:
                        comment_text = comment_el.text.strip()
                        if comment_text:
                            scraped_data["comments"].append(comment_text)
                        # else: # Optional: log empty elements if debugging selector issues
                        #     print(f"Comment element {i+1} had empty text.")
                    except Exception as e_inner:
                        print(f"Could not extract text from comment element {i+1}: {e_inner}")
                print(f"Successfully scraped {len(scraped_data['comments'])} comments.")
            else:
                print("No comment text elements found with the specified XPath, although container was present.")
                driver.save_screenshot("screenshots/debug_no_comment_elements.png")

        except TimeoutException:
            print("Comments container or first comment did not load/appear within timeout.")
            driver.save_screenshot("screenshots/debug_timeout_comments.png")
        except Exception as e:
            print(f"Error scraping comments: {e}")
            driver.save_screenshot("screenshots/debug_error_scraping_comments.png")

    except Exception as e:
        print(f"An critical error occurred: {e}")
        driver.save_screenshot("screenshots/debug_critical_error.png")
    finally:
        print("Closing the browser.")
        #if driver:
            #driver.quit()

    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    data = scrape_xueqiu_post_and_comments(target_url)

    print("\n--- Scraped Data ---")
    print("\nMain Post:")
    print(data["main_post"] if data["main_post"] else "Not found or scraping failed.")
    print("\nComments:")
    if data["comments"]:
        for i, comment in enumerate(data["comments"]):
            print(f"{i+1}. {comment}")
    else:
        print("No comments found or scraping failed.")

Navigating to: https://xueqiu.com/5669998349/334081638
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') did not appear or was not found within timeout.
Taking screenshot before handling second popup...
Screenshot saved to screenshots/debug_before_second_popup.png
Looking for the second pop-up (close button 'X')...
Checking for known iframe patterns...
No iframe found matching pattern, or 'X' button not found inside iframe. Proceeding to check main page.
Looking for close button on the main page...
Trying XPath: //div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]
Close button with XPath '//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]' not found or not clickable yet.
Trying XPath: //div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]
Close button with XPath '//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]' not found or not clickable yet.
Trying XPath: //i[contains(@class, 'cube-dialog-close')]
Close button

In [15]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def debug_scroll_and_screenshot(url, max_scroll_attempts=10):
    """
    Loads Xueqiu page, handles popups, clicks comments, then repeatedly
    scrolls, expands replies, clicks 'load more', taking screenshots.

    Args:
        url (str): The URL of the Xueqiu post.
        max_scroll_attempts (int): Max number of scroll/interact cycles.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # Keep browser visible for debugging
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1200,900") # Moderate size
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_debug_scroll"
    if not os.path.exists(screenshot_dir):
        try:
            os.makedirs(screenshot_dir)
            print(f"Created '{screenshot_dir}' directory.")
        except OSError as e:
            print(f"Error creating screenshot directory: {e}")
            return # Stop if we can't save screenshots

    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        # Use shorter waits for finding elements to click in the loop
        wait = WebDriverWait(driver, 15) # For initial setup
        short_wait = WebDriverWait(driver, 3) # For finding dynamic elements in loop

        print(f"Navigating to: {url}")
        driver.get(url)

        # --- Handle Initial Pop-ups (Reusing previous robust logic) ---
        # First Pop-up ("跳过")
        try:
            print("Looking for the first pop-up ('跳过')...")
            skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"
            skip_button = wait.until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            print("Clicking '跳过'...")
            driver.execute_script("arguments[0].click();", skip_button)
            time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out.")
        except Exception as e: print(f"Error handling first pop-up: {e}")

        # Second Pop-up ("X")
        try:
            print("Looking for the second pop-up ('X')...")
            close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try:
                     close_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
                     print(f"Clicking second pop-up 'X' (XPath: {xpath})...")
                     driver.execute_script("arguments[0].click();", close_button)
                     close_button_found = True
                     time.sleep(0.5)
                     break
                 except TimeoutException: continue
                 except Exception: continue # Ignore other errors for this debug script
            if not close_button_found: print("Second pop-up ('X') not found or timed out.")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # --- Click on Comments Tab ---
        comments_tab_clicked = False
        try:
            print("Looking for the '评论' (Comments) tab...")
            tab_xpath = "//div[contains(@class, 'tabs__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')]"
            comments_tab = wait.until(EC.element_to_be_clickable((By.XPATH, tab_xpath)))
            print("Scrolling '评论' tab into view and clicking...")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", comments_tab)
            time.sleep(0.3)
            driver.execute_script("arguments[0].click();", comments_tab)
            print("Clicked '评论' tab.")
            comments_tab_clicked = True
            time.sleep(1.5) # Wait for initial comments section to load
        except Exception as e:
            print(f"Error finding or clicking '评论' tab: {e}")
            if driver: driver.save_screenshot(os.path.join(screenshot_dir, "error_clicking_comments_tab.png"))

        # --- Main Interaction Loop ---
        if comments_tab_clicked:
            print("\n--- Starting Scroll and Interaction Loop ---")
            last_height = driver.execute_script("return document.body.scrollHeight")

            for i in range(max_scroll_attempts):
                print(f"\n--- Iteration {i+1}/{max_scroll_attempts} ---")
                actions_taken_this_iteration = False

                # 1. Take Screenshot Before Interaction
                screenshot_path = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_before.png")
                try:
                    driver.save_screenshot(screenshot_path)
                    print(f"Saved screenshot: {screenshot_path}")
                except Exception as e:
                    print(f"Failed to save screenshot {screenshot_path}: {e}")

                # 2. Find and Click "Expand Replies" (查看N条回复)
                # Use XPath that finds links/divs containing '查看' and '回复'
                # Make sure they are visible/interactable
                expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '回复')] | //div[contains(@class, 'reply-expander') and contains(., '查看') and contains(., '回复')]"
                try:
                    expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                    print(f"Found {len(expand_buttons)} potential 'Expand Replies' links/buttons.")
                    clicked_count = 0
                    for j, button in enumerate(expand_buttons):
                        try:
                            # Check if it's visible before trying to click
                            if button.is_displayed():
                                print(f"Attempting to click 'Expand Replies' #{j+1}...")
                                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button) # Scroll to button
                                time.sleep(0.2)
                                driver.execute_script("arguments[0].click();", button)
                                print("Clicked.")
                                actions_taken_this_iteration = True
                                clicked_count += 1
                                time.sleep(0.8) # Wait for replies to potentially load
                            # else: # Debugging
                            #    print(f"Button #{j+1} found but not displayed.")
                        except (StaleElementReferenceException):
                            print("Stale element reference for 'Expand Replies', likely disappeared after previous click. Skipping.")
                            continue # Element might have been removed from DOM after a previous click
                        except (ElementClickInterceptedException, ElementNotInteractableException):
                             print("Could not click 'Expand Replies' (intercepted or not interactable). Trying JS click again.")
                             try:
                                 driver.execute_script("arguments[0].click();", button)
                                 print("Clicked via JS retry.")
                                 actions_taken_this_iteration = True
                                 clicked_count += 1
                                 time.sleep(0.8)
                             except Exception as e_retry:
                                 print(f"JS retry failed: {e_retry}. Skipping this button.")
                        except Exception as e_click_reply:
                            print(f"Error clicking 'Expand Replies' #{j+1}: {e_click_reply}")
                    if clicked_count > 0:
                         print(f"Clicked {clicked_count} 'Expand Replies' buttons.")
                         # Take screenshot after expanding
                         ss_path_expand = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_after_expand.png")
                         driver.save_screenshot(ss_path_expand)
                         print(f"Saved screenshot: {ss_path_expand}")

                except Exception as e_find_reply:
                    print(f"Error finding 'Expand Replies' buttons: {e_find_reply}")


                # 3. Scroll Down
                print("Scrolling down...")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5) # Wait for page to potentially load new items after scroll

                # 4. Find and Click "Load More Comments" (展开查看更多 / 加载更多)
                # Use XPath targeting text, check for visibility/clickability
                load_more_xpath = "//*[contains(text(), '展开查看更多') or contains(text(), '加载更多')]"
                try:
                    # Use short wait as it might appear after scrolling
                    load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))
                    print("Found 'Load More' button. Attempting to click...")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button) # Scroll to it
                    time.sleep(0.3)
                    driver.execute_script("arguments[0].click();", load_more_button)
                    print("Clicked 'Load More'.")
                    actions_taken_this_iteration = True
                    time.sleep(2.5) # Wait longer after clicking load more
                    # Take screenshot after load more
                    ss_path_load = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_after_load_more.png")
                    driver.save_screenshot(ss_path_load)
                    print(f"Saved screenshot: {ss_path_load}")

                except TimeoutException:
                    print("'Load More' button not found or not clickable.")
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    print("Could not click 'Load More' (intercepted or not interactable).") # Don't retry here for simplicity
                except Exception as e_load_more:
                    print(f"Error finding/clicking 'Load More': {e_load_more}")

                # 5. Check for End Condition (Simplified)
                current_height = driver.execute_script("return document.body.scrollHeight")
                print(f"Current height: {current_height}, Last height: {last_height}")
                if not actions_taken_this_iteration and current_height == last_height:
                    print("No actions taken and scroll height did not change. Ending loop.")
                    break
                last_height = current_height
            else:
                print(f"\n--- Reached max iterations ({max_scroll_attempts}) ---")

        else:
            print("Skipping interaction loop because '评论' tab was not clicked.")

        print("\n--- Taking final screenshot ---")
        final_ss_path = os.path.join(screenshot_dir, "final_state.png")
        driver.save_screenshot(final_ss_path)
        print(f"Saved final screenshot: {final_ss_path}")

    except Exception as e:
        print(f"\n--- An critical error occurred during the debug process ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved error screenshot: {error_ss_path}")
            except Exception as e_ss:
                 print(f"Could not save error screenshot: {e_ss}")
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")
        else:
            print("Driver was not initialized.")

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print("--- Starting Xueqiu Debug Scroll/Screenshot Script ---")
    debug_scroll_and_screenshot(target_url, max_scroll_attempts=8) # Adjust max attempts as needed
    print("--- Debug Script Finished ---")
    print(f"Check the 'screenshots_debug_scroll' folder.")

--- Starting Xueqiu Debug Scroll/Screenshot Script ---
Setting up WebDriver...
Created 'screenshots_debug_scroll' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') not found or timed out.
Looking for the second pop-up ('X')...
Second pop-up ('X') not found or timed out.
Looking for the '评论' (Comments) tab...
Error finding or clicking '评论' tab: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128A145+76773]
	GetHandleVerifier [0x00007FF7E128A1A0+76864]
	(No symbol) [0x00007FF7E1048F7A]
	(No symbol) [0x00007FF7E109F496]
	(No symbol) [0x00007FF7E109F74C]
	(No symbol) [0x00007FF7E10F2287]
	(No symbol) [0x00007FF7E10C739F]
	(No symbol) [0x00007FF7E10EF0CF]
	(No symbol) [0x00007FF7E10C7133]
	(No symbol) [0x00007FF7E10904D1]
	(No symbol) [0x00007FF7E1091263]
	GetHandleVerifier [0x00007FF7E154A8ED+2962317]
	GetHandleVerifier [0x00007FF7E1544EC2+2939234]
	GetHandleVerifier [0x00007FF7E1562FF3+3062419]
	GetHandleVerif

In [17]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def debug_scroll_and_screenshot(url, max_scroll_attempts=10):
    """
    Loads Xueqiu page, handles popups, clicks comments, then repeatedly
    scrolls, expands replies, clicks 'load more', taking screenshots.

    Args:
        url (str): The URL of the Xueqiu post.
        max_scroll_attempts (int): Max number of scroll/interact cycles.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # Keep browser visible for debugging
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_debug_scroll"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return

    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        # INCREASED Wait time for critical elements like the tab
        wait = WebDriverWait(driver, 25) # Increased to 25 seconds
        short_wait = WebDriverWait(driver, 5) # For loop interactions

        print(f"Navigating to: {url}")
        driver.get(url)
        time.sleep(2) # Add a small static pause after initial load

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for the first pop-up ('跳过')..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            print("Clicking '跳过'..."); driver.execute_script("arguments[0].click();", skip_button); time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out.")
        except Exception as e: print(f"Error handling first pop-up: {e}")
        try:
            print("Looking for the second pop-up ('X')..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath))); print(f"Clicking second pop-up 'X' (XPath: {xpath})..."); driver.execute_script("arguments[0].click();", close_button); close_button_found = True; time.sleep(0.5); break
                 except TimeoutException: continue
                 except Exception: continue
            if not close_button_found: print("Second pop-up ('X') not found or timed out.")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # --- Click on Comments Tab ---
        comments_tab_clicked = False
        try:
            print("Looking for the '评论' (Comments) tab...")
            tab_xpath = "//span[text()='评论' and contains(@class,'tabs__item__title')]/ancestor::div[contains(@class,'tabs__item')] | //div[contains(@class, 'tabs__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')]"
            print(f"Using Tab XPath: {tab_xpath}")

            print("Waiting for tab presence...")
            comments_tab_element = wait.until(EC.presence_of_element_located((By.XPATH, tab_xpath)))
            print("Tab present. Scrolling tab into view...")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", comments_tab_element)
            time.sleep(0.5)

            print("Waiting for tab to be clickable...")
            comments_tab_clickable = wait.until(EC.element_to_be_clickable((By.XPATH, tab_xpath)))
            print("Tab clickable. Clicking '评论' tab...")
            driver.execute_script("arguments[0].click();", comments_tab_clickable)
            print("Clicked '评论' tab.")
            comments_tab_clicked = True
            time.sleep(2.0)

        except TimeoutException as e_timeout:
             print(f"TimeoutException finding or clicking '评论' tab: {e_timeout.msg}")
             if driver:
                  screenshot_path = os.path.join(screenshot_dir, "error_timeout_comments_tab.png")
                  driver.save_screenshot(screenshot_path)
                  print(f"Saved error screenshot: {screenshot_path}")
        except Exception as e:
            print(f"General Error finding or clicking '评论' tab: {e}")
            if driver:
                 screenshot_path = os.path.join(screenshot_dir, "error_generic_comments_tab.png")
                 driver.save_screenshot(screenshot_path)
                 print(f"Saved error screenshot: {screenshot_path}")

        # --- Main Interaction Loop ---
        if comments_tab_clicked:
            print("\n--- Starting Scroll and Interaction Loop ---")
            last_height = driver.execute_script("return document.body.scrollHeight")
            for i in range(max_scroll_attempts):
                print(f"\n--- Iteration {i+1}/{max_scroll_attempts} ---")
                actions_taken_this_iteration = False
                # 1. Screenshot Before
                screenshot_path = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_before.png"); driver.save_screenshot(screenshot_path); print(f"Saved screenshot: {screenshot_path}")
                # 2. Click "Expand Replies"
                expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '回复')] | //div[contains(@class, 'reply-expander') and contains(., '查看') and contains(., '回复')]"
                try:
                    expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath); print(f"Found {len(expand_buttons)} 'Expand Replies'.")
                    clicked_count = 0
                    for j, button in enumerate(expand_buttons):
                        try:
                            if button.is_displayed():
                                print(f"Clicking 'Expand Replies' #{j+1}..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button); time.sleep(0.2)
                                driver.execute_script("arguments[0].click();", button); print("Clicked."); actions_taken_this_iteration = True; clicked_count += 1; time.sleep(0.8)
                        except (StaleElementReferenceException): print("Stale element on 'Expand Replies', skipping."); continue
                        except (ElementClickInterceptedException, ElementNotInteractableException):
                             print("Click intercepted/not interactable on 'Expand Replies'. Trying JS retry.");
                             try: driver.execute_script("arguments[0].click();", button); print("Clicked via JS retry."); actions_taken_this_iteration = True; clicked_count += 1; time.sleep(0.8)
                             except Exception as e_retry: print(f"JS retry failed: {e_retry}. Skipping.")
                        except Exception as e_click_reply: print(f"Error clicking 'Expand Replies' #{j+1}: {e_click_reply}")
                    if clicked_count > 0: print(f"Clicked {clicked_count} 'Expand Replies'."); ss_path_expand = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_after_expand.png"); driver.save_screenshot(ss_path_expand); print(f"Saved screenshot: {ss_path_expand}")
                except Exception as e_find_reply: print(f"Error finding 'Expand Replies': {e_find_reply}")
                # 3. Scroll Down
                print("Scrolling down..."); driver.execute_script("window.scrollTo(0, document.body.scrollHeight);"); time.sleep(1.5)
                # 4. Click "Load More Comments"
                load_more_xpath = "//*[contains(text(), '展开查看更多') or contains(text(), '加载更多')]"
                try:
                    load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))
                    print("Clicking 'Load More'..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button); time.sleep(0.3)
                    driver.execute_script("arguments[0].click();", load_more_button); print("Clicked 'Load More'."); actions_taken_this_iteration = True; time.sleep(2.5)
                    ss_path_load = os.path.join(screenshot_dir, f"scroll_iter_{i+1}_after_load_more.png"); driver.save_screenshot(ss_path_load); print(f"Saved screenshot: {ss_path_load}")
                except TimeoutException: print("'Load More' button not found or not clickable.")
                except (ElementClickInterceptedException, ElementNotInteractableException): print("Could not click 'Load More'.")
                except Exception as e_load_more: print(f"Error clicking 'Load More': {e_load_more}")
                # 5. Check End Condition
                current_height = driver.execute_script("return document.body.scrollHeight"); print(f"Current height: {current_height}, Last height: {last_height}")
                if not actions_taken_this_iteration and current_height == last_height: print("No actions taken and scroll height did not change. Ending loop."); break
                last_height = current_height
            else: print(f"\n--- Reached max iterations ({max_scroll_attempts}) ---")
        else: print("Skipping interaction loop because '评论' tab was not successfully clicked.")

        print("\n--- Taking final screenshot ---")
        final_ss_path = os.path.join(screenshot_dir, "final_state.png"); driver.save_screenshot(final_ss_path); print(f"Saved final screenshot: {final_ss_path}")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # --- CORRECTED INDENTATION FOR ERROR SCREENSHOT TRY-EXCEPT ---
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved error screenshot: {error_ss_path}")
            except Exception as e_ss:
                 print(f"Could not save error screenshot: {e_ss}")
        # --- END CORRECTION ---
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")
        else:
            print("Driver was not initialized.")
    # Removed return as function doesn't explicitly return anything

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print("--- Starting Xueqiu Debug Scroll/Screenshot Script ---")
    debug_scroll_and_screenshot(target_url, max_scroll_attempts=8)
    print("--- Debug Script Finished ---")
    print(f"Check the '{screenshot_dir}' folder.")

--- Starting Xueqiu Debug Scroll/Screenshot Script ---
Setting up WebDriver...
Navigating to: https://xueqiu.com/5669998349/334081638
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') not found or timed out.
Looking for the second pop-up ('X')...
Second pop-up ('X') not found or timed out.
Looking for the '评论' (Comments) tab...
Using Tab XPath: //span[text()='评论' and contains(@class,'tabs__item__title')]/ancestor::div[contains(@class,'tabs__item')] | //div[contains(@class, 'tabs__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')]
Waiting for tab presence...
TimeoutException finding or clicking '评论' tab: 
Saved error screenshot: screenshots_debug_scroll\error_timeout_comments_tab.png
Skipping interaction loop because '评论' tab was not successfully clicked.

--- Taking final screenshot ---
Saved final screenshot: screenshots_debug_scroll\final_state.png
Closing the browser...
Browser closed.
--- Debug Script Finished ---


NameError: name 'screenshot_dir' is not defined

In [19]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys # Needed for Enter key
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException
)

def search_and_scrape_xueqiu(keyword, max_results=20):
    """
    Searches Xueqiu for a keyword and scrapes post titles and links from results.

    Args:
        keyword (str): The search term.
        max_results (int): Approximate maximum number of results to scrape.

    Returns:
        list: A list of dictionaries, each containing 'title' and 'link'.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # Keep browser visible for debugging
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_search"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return []

    driver = None
    scraped_results = []

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20) # Increased wait time
        short_wait = WebDriverWait(driver, 5)

        print(f"Navigating to: https://xueqiu.com/")
        driver.get("https://xueqiu.com/")
        time.sleep(2)

        # --- Handle Initial Pop-ups (Reusing previous robust logic) ---
        try:
            print("Looking for the first pop-up ('跳过')..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            print("Clicking '跳过'..."); driver.execute_script("arguments[0].click();", skip_button); time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out.")
        except Exception as e: print(f"Error handling first pop-up: {e}")
        try:
            print("Looking for the second pop-up ('X')..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath))); print(f"Clicking second pop-up 'X'..."); driver.execute_script("arguments[0].click();", close_button); close_button_found = True; time.sleep(0.5); break
                 except TimeoutException: continue
                 except Exception: continue
            if not close_button_found: print("Second pop-up ('X') not found or timed out.")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # --- Find Search Bar and Search ---
        try:
            print(f"Looking for the search bar...")
            # Locator for the search input, often has a placeholder
            search_input_xpath = "//input[@placeholder='搜索股票/组合/用户']"
            search_input = wait.until(EC.visibility_of_element_located((By.XPATH, search_input_xpath)))
            print(f"Search bar found. Typing keyword: '{keyword}'")
            search_input.send_keys(keyword)
            time.sleep(0.5) # Brief pause after typing
            print("Submitting search (pressing Enter)...")
            search_input.send_keys(Keys.ENTER)

            # --- Wait for Search Results Page ---
            print("Waiting for search results page to load...")
            # Wait for a container that holds search results
            # Inspect the page after searching to find a reliable container ID/class
            # Common patterns: 'search-result-list', 'result-list', 'timeline' section
            results_container_xpath = "//div[contains(@class, 'search__result')] | //div[contains(@class, 'timeline')]" # Adjust based on inspection
            wait.until(EC.presence_of_element_located((By.XPATH, results_container_xpath)))
            print("Search results container found.")
            # Add a small pause for results within the container to render
            time.sleep(2)

            # --- Scrape Results ---
            print("Scraping search results...")
            # Locator for individual result items (adjust based on inspection)
            # Often divs or articles with classes like 'search__result__item', 'timeline__item'
            result_item_xpath = "//div[contains(@class, 'search__result__item')] | //article[contains(@class, 'timeline__item')]"
            result_items = driver.find_elements(By.XPATH, result_item_xpath)

            print(f"Found {len(result_items)} potential result items on the page.")

            count = 0
            for item in result_items:
                if count >= max_results:
                    print(f"Reached max results limit ({max_results}).")
                    break
                try:
                    # Find the title link within the item (adjust locator)
                    # Often an 'a' tag inside a title div/heading
                    title_link_element = item.find_element(By.XPATH, ".//a[contains(@class, 'title')] | .//div[contains(@class,'timeline__item__content')]//a[contains(@href,'/')]") # Relative XPath

                    title = title_link_element.text.strip()
                    link = title_link_element.get_attribute('href')

                    # Basic validation and filtering
                    if title and link and link.startswith("http"): # Ensure it's a valid link
                        print(f"  - Found: {title} ({link})")
                        scraped_results.append({"title": title, "link": link})
                        count += 1
                    elif title and link: # Handle relative links if necessary
                        print(f"  - Found (relative link): {title} ({link})")
                        # You might need to join this with the base URL if needed
                        scraped_results.append({"title": title, "link": link})
                        count += 1

                except NoSuchElementException:
                    # print("  - Could not find title/link in one item, skipping.") # Can be verbose
                    continue # Skip this item if structure is different
                except Exception as e_item:
                    print(f"  - Error processing one item: {e_item}")
                    continue

            if not scraped_results:
                print("No results scraped. Check locators or if results loaded.")
                driver.save_screenshot(os.path.join(screenshot_dir, "no_results_scraped.png"))

        except TimeoutException as e_search:
            print(f"Timeout error during search or results loading: {e_search.msg}")
            driver.save_screenshot(os.path.join(screenshot_dir, "error_search_timeout.png"))
        except Exception as e:
            print(f"An error occurred during search or scraping: {e}")
            driver.save_screenshot(os.path.join(screenshot_dir, "error_search_generic.png"))

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error: {e}")
        if driver:
            try: driver.save_screenshot(os.path.join(screenshot_dir, "critical_error.png"))
            except: pass
    finally:
        if driver:
           # print("Closing the browser...")
           # driver.quit()
            print("Browser closed.")

    return scraped_results

# --- Main execution block ---
if __name__ == "__main__":
    search_keyword = "公平"
    print(f"--- Starting Xueqiu Search Scraper for keyword: '{search_keyword}' ---")
    results = search_and_scrape_xueqiu(search_keyword, max_results=15)
    print("--- Scraping Process Finished ---")

    print("\n" + "="*30)
    print("      Scraped Search Results")
    print("="*30)

    if results:
        print(f"Scraped {len(results)} results:")
        for i, result in enumerate(results):
            print(f"{i+1}. Title: {result['title']}")
            print(f"   Link: {result['link']}")
    else:
        print(">>> No results were successfully scraped. <<<")

    print("\n" + "="*30)
    print(f"Check console logs and folder for details.")

--- Starting Xueqiu Search Scraper for keyword: '公平' ---
Setting up WebDriver...
Navigating to: https://xueqiu.com/
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') not found or timed out.
Looking for the second pop-up ('X')...
Second pop-up ('X') not found or timed out.
Looking for the search bar...
Browser closed.


KeyboardInterrupt: 

In [20]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException
)

def search_and_scrape_xueqiu_revised(keyword, max_results=20):
    """
    Searches Xueqiu homepage, clicks search button, and scrapes results.

    Args:
        keyword (str): The search term.
        max_results (int): Approximate maximum number of results to scrape.

    Returns:
        list: A list of dictionaries, each containing 'title' and 'link'.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1366,768") # Common size
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_search"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return []

    driver = None
    scraped_results = []

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window() # Still maximize

        wait = WebDriverWait(driver, 20)
        short_wait = WebDriverWait(driver, 5)

        print(f"Navigating to: https://xueqiu.com/")
        driver.get("https://xueqiu.com/")
        time.sleep(2) # Allow initial load

        # --- Handle Initial Pop-ups (Keep trying, may not appear) ---
        try:
            print("Looking for the first pop-up ('跳过')..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 7).until(EC.element_to_be_clickable((By.XPATH, skip_xpath))) # Shorter wait
            print("Clicking '跳过'..."); driver.execute_script("arguments[0].click();", skip_button); time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out (might be expected).")
        except Exception as e: print(f"Error handling first pop-up: {e}")
        try:
            print("Looking for the second pop-up ('X')..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, xpath))); print(f"Clicking second pop-up 'X'..."); driver.execute_script("arguments[0].click();", close_button); close_button_found = True; time.sleep(0.5); break
                 except TimeoutException: continue
                 except Exception: continue
            if not close_button_found: print("Second pop-up ('X') not found or timed out (might be expected).")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # --- Find Search Bar and Search ---
        try:
            print(f"Looking for the search bar...")
            search_input_xpath = "//input[@placeholder='搜索股票/组合/用户']"
            search_button_xpath = "//div[contains(@class,'navbar__search')]//i[contains(@class,'search-icon')]/parent::div" # Find icon, go to parent div which is clickable

            search_input = wait.until(EC.element_to_be_clickable((By.XPATH, search_input_xpath)))
            print(f"Search bar found. Clicking and typing keyword: '{keyword}'")
            # search_input.click() # Click first sometimes helps
            # time.sleep(0.2)
            search_input.clear() # Clear any potential default text
            search_input.send_keys(keyword)
            time.sleep(0.5)

            print("Looking for search button...")
            search_button = wait.until(EC.element_to_be_clickable((By.XPATH, search_button_xpath)))
            print("Search button found. Clicking search button...")
            search_button.click()

            # --- Wait for Search Results Page ---
            print("Waiting for search results page transition...")
            # Option 1: Wait for URL change
            wait.until(EC.url_contains("/search?keyword="))
            print(f"URL changed, indicating search results page loaded: {driver.current_url}")
            # Option 2: Wait for a specific element unique to the results page
            # results_page_marker_xpath = "//div[contains(@class,'result-title-wrap')]" # Adjust this! Inspect results page for a unique static element
            # wait.until(EC.presence_of_element_located((By.XPATH, results_page_marker_xpath)))
            # print("Results page marker element found.")

            # Save screenshot of results page
            time.sleep(2) # Allow results rendering
            results_screenshot_path = os.path.join(screenshot_dir, "search_results_page.png")
            driver.save_screenshot(results_screenshot_path)
            print(f"Screenshot of search results page saved to: {results_screenshot_path}")

            # --- Scrape Results ---
            print("Scraping search results...")
            # *** THESE LOCATORS MUST BE ADJUSTED BASED ON ACTUAL RESULTS PAGE INSPECTION ***
            # Example based on common structures, VERY LIKELY NEEDS CORRECTION
            result_item_xpath = "//div[@class='search-item']" # Example results item wrapper
            title_link_relative_xpath = ".//a[contains(@class,'title')]" # Example title link within item

            # Find items using the potentially updated item xpath
            result_items = driver.find_elements(By.XPATH, result_item_xpath)
            print(f"Found {len(result_items)} potential result items using XPath: {result_item_xpath}")

            count = 0
            if not result_items:
                print("No result items found using the current XPath. Please inspect the page and update 'result_item_xpath'.")

            for item in result_items:
                if count >= max_results:
                    print(f"Reached max results limit ({max_results}).")
                    break
                try:
                    # Find link/title using the potentially updated relative xpath
                    title_link_element = item.find_element(By.XPATH, title_link_relative_xpath)
                    title = title_link_element.text.strip()
                    link = title_link_element.get_attribute('href')

                    if title and link and link.startswith("http"):
                        print(f"  - Found: {title} ({link})")
                        scraped_results.append({"title": title, "link": link})
                        count += 1
                    elif title and link:
                        # Handle relative URLs if needed by joining with base URL
                        # from urllib.parse import urljoin
                        # base_url = "https://xueqiu.com"
                        # absolute_link = urljoin(base_url, link)
                        # print(f"  - Found (relative): {title} ({absolute_link})")
                        # scraped_results.append({"title": title, "link": absolute_link})
                        print(f"  - Found (relative/fragment?): {title} ({link})")
                        scraped_results.append({"title": title, "link": link}) # Store as is for now
                        count += 1

                except NoSuchElementException:
                    print("  - Could not find title/link in one item using relative XPath, skipping. Please inspect and update 'title_link_relative_xpath'.")
                    continue
                except Exception as e_item:
                    print(f"  - Error processing one item: {e_item}")
                    continue

            if not scraped_results and result_items:
                 print("Found result item containers, but failed to extract title/link. Check 'title_link_relative_xpath'.")

            if not scraped_results and not result_items:
                print("No results scraped. Check results page structure and locators.")
                # Screenshot already taken, but could take another one here if needed

        except TimeoutException as e_search:
            print(f"Timeout error during search or results loading: {e_search.msg}")
            if driver: driver.save_screenshot(os.path.join(screenshot_dir, "error_search_timeout.png"))
        except Exception as e:
            print(f"An error occurred during search or scraping: {e}")
            if driver: driver.save_screenshot(os.path.join(screenshot_dir, "error_search_generic.png"))

    except Exception as e:
        print(f"\n--- An critical error occurred ---"); print(f"Error: {e}")
        if driver: try: driver.save_screenshot(os.path.join(screenshot_dir, "critical_error.png"))
                   except: pass
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")

    return scraped_results

# --- Main execution block ---
if __name__ == "__main__":
    search_keyword = "公平"
    print(f"--- Starting Xueqiu Search Scraper for keyword: '{search_keyword}' ---")
    results = search_and_scrape_xueqiu_revised(search_keyword, max_results=15)
    print("--- Scraping Process Finished ---")

    print("\n" + "="*30); print("      Scraped Search Results"); print("="*30)
    if results:
        print(f"Scraped {len(results)} results:")
        for i, result in enumerate(results): print(f"{i+1}. Title: {result['title']}\n   Link: {result['link']}")
    else: print(">>> No results were successfully scraped. <<<")
    print("\n" + "="*30); print(f"Check console logs and '{screenshot_dir}' folder for details.")

SyntaxError: invalid syntax (3608465887.py, line 168)

In [22]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException
)

def scroll_and_screenshot_by_distance(url, scroll_pause_time=1.5, scroll_increment_js="window.scrollBy(0, window.innerHeight);"):
    """
    Navigates to a URL, handles popups, scrolls down in increments, taking a screenshot
    at each step until the bottom of the page is reached.

    Args:
        url (str): The URL of the page to scroll.
        scroll_pause_time (float): Time in seconds to wait after each scroll
                                   for content to potentially load.
        scroll_increment_js (str): JavaScript to execute for scrolling.
                                   Default scrolls by one viewport height.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # Keep browser visible to observe
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,800")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_incremental_scroll"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return

    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        # driver.maximize_window() # Maximize or set specific size above

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("Body element loaded.")
        except TimeoutException:
            print("Page body did not become present within timeout.")
        time.sleep(3)

        # --- Handle Initial Pop-ups (Integrated robust logic) ---
        try:
            print("Looking for the first pop-up ('跳过')..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            print("Clicking '跳过'..."); driver.execute_script("arguments[0].click();", skip_button); time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out.")
        except Exception as e: print(f"Error handling first pop-up: {e}")
        try:
            print("Looking for the second pop-up ('X')..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath))); print(f"Clicking second pop-up 'X'..."); driver.execute_script("arguments[0].click();", close_button); close_button_found = True; time.sleep(0.5); break
                 except TimeoutException: continue
                 except Exception: continue
            if not close_button_found: print("Second pop-up ('X') not found or timed out.")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # Optional: Click 'Comments' Tab
        # try:
        #     print("Looking for and clicking '评论' tab before scrolling...")
        #     tab_xpath = "//span[text()='评论' and contains(@class,'tabs__item__title')]/ancestor::div[contains(@class,'tabs__item')] | //div[contains(@class, 'tabs__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')]"
        #     comments_tab_element = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, tab_xpath)))
        #     driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", comments_tab_element)
        #     time.sleep(0.5)
        #     comments_tab_clickable = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, tab_xpath)))
        #     driver.execute_script("arguments[0].click();", comments_tab_clickable)
        #     print("Clicked '评论' tab. Waiting before scroll...")
        #     time.sleep(2.5)
        # except Exception as e_tab:
        #     print(f"Could not click '评论' tab before scrolling (maybe not needed or error): {e_tab}")

        print("\n--- Starting Incremental Scroll and Screenshot Process ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_attempt = 0

        while True:
            scroll_attempt += 1
            print(f"--- Scroll Attempt #{scroll_attempt} ---")

            screenshot_path = os.path.join(screenshot_dir, f"scroll_{scroll_attempt:02d}_before.png")
            try:
                driver.save_screenshot(screenshot_path)
                print(f"Saved screenshot: {screenshot_path}")
            except Exception as e_ss:
                print(f"Failed to save screenshot {screenshot_path}: {e_ss}")

            print(f"Scrolling down using: {scroll_increment_js}")
            driver.execute_script(scroll_increment_js)

            print(f"Pausing for {scroll_pause_time} seconds...")
            time.sleep(scroll_pause_time)

            new_height = driver.execute_script("return document.body.scrollHeight")
            current_scroll_y = driver.execute_script("return window.pageYOffset || document.documentElement.scrollTop;")
            viewport_height = driver.execute_script("return window.innerHeight;")
            print(f"  Current scrollY: {round(current_scroll_y)}, Viewport height: {round(viewport_height)}, Total scrollHeight: {new_height}")

            if current_scroll_y + viewport_height >= new_height - 10:
                print("Reached the bottom of the page.")
                final_bottom_path = os.path.join(screenshot_dir, f"scroll_{scroll_attempt:02d}_at_bottom.png")
                try: driver.save_screenshot(final_bottom_path)
                except Exception as e_fin_ss: print(f"Could not save final screenshot: {e_fin_ss}")
                print(f"Saved final bottom screenshot: {final_bottom_path}")
                break
            elif scroll_attempt > 50:
                print("Reached max scroll attempts (50). Stopping.")
                break
            last_height = new_height
        print("\n--- Incremental Scroll and Screenshot Process Finished ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # --- CORRECTED INDENTATION FOR ERROR SCREENSHOT TRY-EXCEPT ---
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error_scroll_script.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved error screenshot: {error_ss_path}")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
        # --- END CORRECTION ---
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Incremental Scroll for URL: {target_url} ---")
    scroll_and_screenshot_by_distance(
        target_url,
        scroll_pause_time=2.0,
        scroll_increment_js="window.scrollBy(0, window.innerHeight * 0.8);"
        )
    print("--- Script Finished ---")
    print(f"Check the '{screenshot_dir}' folder for screenshots.")

--- Starting Incremental Scroll for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_incremental_scroll' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Body element loaded.
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') not found or timed out.
Looking for the second pop-up ('X')...
Second pop-up ('X') not found or timed out.

--- Starting Incremental Scroll and Screenshot Process ---
--- Scroll Attempt #1 ---
Saved screenshot: screenshots_incremental_scroll\scroll_01_before.png
Scrolling down using: window.scrollBy(0, window.innerHeight * 0.8);
Pausing for 2.0 seconds...
  Current scrollY: 482, Viewport height: 602, Total scrollHeight: 4924
--- Scroll Attempt #2 ---
Saved screenshot: screenshots_incremental_scroll\scroll_02_before.png
Scrolling down using: window.scrollBy(0, window.innerHeight * 0.8);
Pausing for 2.0 seconds...
  Current scrollY: 963, Viewport height: 602, Total scrollHeight: 4924
--- Scroll At

NameError: name 'screenshot_dir' is not defined

In [25]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException
)

def scrape_post_and_comments_on_scroll(url, max_scroll_loops=15, scroll_pause_time=2.5):
    """
    Navigates to a Xueqiu post, handles popups, scrolls to load comments,
    and attempts to scrape them. Also clicks 'expand replies' and 'load more'.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900") # Wider for layout
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    # Use a set to store scraped comment texts to automatically handle duplicates
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20) # For initial elements
        short_wait = WebDriverWait(driver, 7) # For dynamic elements in loop

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load. Page might be different."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2) # Pause for popups

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue # Try next
        except Exception as e: print(f"Error 'X' pop-up: {e}")
        # --- End Pop-up Handling ---

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))


        print("\n--- Starting scroll and comment extraction ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        no_new_content_strikes = 0

        for i in range(max_scroll_loops):
            print(f"--- Loop/Scroll attempt #{i+1} ---")
            initial_comment_count = len(unique_comment_texts_scraped)

            # 1. Click "查看N条回复" (Expand Replies) - these often appear within comment threads
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            try:
                # Find all such links that are currently visible
                expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                if expand_buttons:
                    print(f"Found {len(expand_buttons)} 'Expand Replies' links.")
                    for button_idx, button in enumerate(expand_buttons):
                        try:
                            if button.is_displayed(): # Click only if visible
                                print(f"  Clicking 'Expand Replies' #{button_idx+1}...")
                                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                                time.sleep(0.3)
                                driver.execute_script("arguments[0].click();", button)
                                time.sleep(1.5) # Wait for replies to load
                        except StaleElementReferenceException: print("  Stale 'Expand Replies' link, skipping.")
                        except ElementNotInteractableException: print("  'Expand Replies' link not interactable, skipping.")
                        except Exception as e_expand: print(f"  Error clicking 'Expand Replies': {e_expand}")
            except Exception as e_find_expand:
                print(f"Could not search for 'Expand Replies' buttons: {e_find_expand}")


            # 2. Scroll down
            print("Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 500);") # Scroll a bit past current perceived bottom
            time.sleep(scroll_pause_time) # Wait for new content

            # 3. Attempt to scrape comments
            # XPath based on your "Inspect Element" screenshot: <div class="comment__item__main"><p>...</p></div>
            comment_text_xpath = "//div[@class='comment__item__main']/p"
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                if comment_p_tags:
                    print(f"  Found {len(comment_p_tags)} potential comment <p> tags on this pass.")
                    new_comments_found_this_pass = 0
                    for p_tag in comment_p_tags:
                        try:
                            comment_text = p_tag.text.strip()
                            if comment_text and comment_text not in unique_comment_texts_scraped:
                                # print(f"    New Comment: {comment_text[:70]}...") # Can be verbose
                                unique_comment_texts_scraped.add(comment_text)
                                new_comments_found_this_pass +=1
                        except StaleElementReferenceException:
                            # print("    Stale comment p_tag, skipping.")
                            continue # Element removed from DOM, common during dynamic updates
                        except Exception as e_text:
                            print(f"    Error getting text from a p_tag: {e_text}")
                    if new_comments_found_this_pass > 0:
                        print(f"    Added {new_comments_found_this_pass} new unique comments this pass.")
            except Exception as e_find:
                print(f"  Error finding comment <p> tags: {e_find}")


            # 4. Click "展开查看更多" (Load More Main Comments) - usually at the very bottom of comment list
            load_more_comments_xpath = "//div[contains(@class,'more-comment') and contains(., '展开查看更多')]" # Adjust if needed
            try:
                load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多' button. Clicking...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("  Clicked '展开查看更多'.")
                time.sleep(scroll_pause_time) # Wait for more comments
            except TimeoutException:
                # print("  '展开查看更多' button not found or not clickable on this pass.") # Expected if no more
                pass
            except Exception as e_load_more:
                print(f"  Error clicking '展开查看更多': {e_load_more}")


            # 5. Check for loop termination conditions
            current_height = driver.execute_script("return document.body.scrollHeight")
            print(f"  Current total comments scraped: {len(unique_comment_texts_scraped)}. Scroll height: {current_height}")
            if len(unique_comment_texts_scraped) > initial_comment_count:
                no_new_content_strikes = 0 # Reset if new comments were found
            else:
                no_new_content_strikes += 1

            if current_height == last_height and no_new_content_strikes >= 2 : # If height hasn't changed for 2 loops AND no new comments
                print("Scroll height unchanged and no new comments for 2 strikes. Assuming all loaded or stuck.")
                break
            elif no_new_content_strikes >= 3: # If no new comments for 3 loops even if height changes (e.g. ads loading)
                print("No new comments found for 3 consecutive strikes. Assuming all loaded.")
                break

            last_height = current_height
            if i == max_scroll_loops -1 :
                print("Reached max scroll loops.")

            # Screenshot at end of loop
            driver.save_screenshot(os.path.join(screenshot_dir,f"loop_end_{i+1}.png"))


        scraped_data["comments"] = list(unique_comment_texts_scraped) # Convert set to list for output
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---"); print(f"Error: {e}")
        if driver: driver.save_screenshot(os.path.join(screenshot_dir, "critical_error.png"))
                   #except: pass
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_comments_on_scroll(target_url, max_scroll_loops=10, scroll_pause_time=3.0)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(data["post_content"][:500] + "..." if len(data["post_content"]) > 500 else data["post_content"])
    else:
        print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"][:20]): # Print first 20
            print(f"{i+1}. {comment[:150]}" + "..." if len(comment)>150 else comment)
        if len(data["comments"]) > 20:
            print(f"... and {len(data['comments']) - 20} more comments.")
    else:
        print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30)
    print(f"Check console logs and '{screenshot_dir}' folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_post_comments' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Scraping main post content...
Post content scraped (length: 627).

--- Starting scroll and comment extraction ---
--- Loop/Scroll attempt #1 ---
Scrolling down...
  Found 18 potential comment <p> tags on this pass.
    Added 2 new unique comments this pass.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #2 ---
Scrolling down...
  Found 18 potential comment <p> tags on this pass.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #3 ---
Scrolling down...
  Found 18 potential comment <p> tags on this pass.
  Current total comments scraped: 2. Scroll height: 4924
Scroll height unchanged and no new comments for 2 st

NameError: name 'screenshot_dir' is not defined

In [26]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException
)

def scrape_post_and_comments_on_scroll(url, max_scroll_loops=15, scroll_pause_time=2.5):
    """
    Navigates to a Xueqiu post, handles popups, scrolls to load comments,
    and attempts to scrape them. Also clicks 'expand replies' and 'load more'.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        short_wait = WebDriverWait(driver, 7)

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load. Page might be different."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.") # Confirmation
        except Exception as e: print(f"Error 'X' pop-up: {e}")
        # --- End Pop-up Handling ---

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))

        print("\n--- Starting scroll and comment extraction ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        no_new_content_strikes = 0

        for i in range(max_scroll_loops):
            print(f"--- Loop/Scroll attempt #{i+1} ---")
            initial_comment_count = len(unique_comment_texts_scraped)

            # 1. Click "查看N条回复"
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            try:
                expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                if expand_buttons:
                    print(f"Found {len(expand_buttons)} 'Expand Replies' links.")
                    for button_idx, button in enumerate(expand_buttons):
                        try:
                            if button.is_displayed():
                                print(f"  Clicking 'Expand Replies' #{button_idx+1}..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button); time.sleep(0.3)
                                driver.execute_script("arguments[0].click();", button); time.sleep(1.5)
                        except StaleElementReferenceException: print("  Stale 'Expand Replies' link, skipping.")
                        except ElementNotInteractableException: print("  'Expand Replies' link not interactable, skipping.")
                        except Exception as e_expand: print(f"  Error clicking 'Expand Replies': {e_expand}")
            except Exception as e_find_expand: print(f"Could not search for 'Expand Replies' buttons: {e_find_expand}")

            # 2. Scroll down
            print("Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 500);")
            time.sleep(scroll_pause_time)

            # 3. Attempt to scrape comments
            comment_text_xpath = "//div[@class='comment__item__main']/p"
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                if comment_p_tags:
                    print(f"  Found {len(comment_p_tags)} potential comment <p> tags.")
                    new_comments_found_this_pass = 0
                    for p_tag in comment_p_tags:
                        try:
                            comment_text = p_tag.text.strip()
                            if comment_text and comment_text not in unique_comment_texts_scraped:
                                unique_comment_texts_scraped.add(comment_text)
                                new_comments_found_this_pass +=1
                        except StaleElementReferenceException: continue
                        except Exception as e_text: print(f"    Error getting text from a p_tag: {e_text}")
                    if new_comments_found_this_pass > 0: print(f"    Added {new_comments_found_this_pass} new unique comments.")
            except Exception as e_find: print(f"  Error finding comment <p> tags: {e_find}")

            # 4. Click "展开查看更多"
            load_more_comments_xpath = "//div[contains(@class,'more-comment') and (contains(., '展开查看更多') or contains(., '加载更多'))]"
            try:
                load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多/加载更多' button. Clicking..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button); time.sleep(0.3)
                driver.execute_script("arguments[0].click();", load_more_button); print("  Clicked '展开查看更多/加载更多'."); time.sleep(scroll_pause_time)
            except TimeoutException: pass
            except Exception as e_load_more: print(f"  Error clicking '展开查看更多': {e_load_more}")

            # 5. Check for loop termination conditions
            current_height = driver.execute_script("return document.body.scrollHeight")
            print(f"  Current total comments scraped: {len(unique_comment_texts_scraped)}. Scroll height: {current_height}")
            if len(unique_comment_texts_scraped) > initial_comment_count: no_new_content_strikes = 0
            else: no_new_content_strikes += 1

            if current_height == last_height and no_new_content_strikes >= 2 :
                print("Scroll height unchanged and no new comments for 2 strikes. Assuming all loaded or stuck.")
                break
            elif no_new_content_strikes >= 3:
                print("No new comments found for 3 consecutive strikes. Assuming all loaded.")
                break
            last_height = current_height
            if i == max_scroll_loops -1 : print("Reached max scroll loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"loop_end_{i+1}.png"))

        scraped_data["comments"] = list(unique_comment_texts_scraped)
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # --- CORRECTED INDENTATION FOR ERROR SCREENSHOT TRY-EXCEPT ---
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
        # --- END CORRECTION ---
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_comments_on_scroll(target_url, max_scroll_loops=10, scroll_pause_time=3.0)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(data["post_content"][:500] + "..." if len(data["post_content"]) > 500 else data["post_content"])
    else:
        print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"][:20]): # Print first 20
            print(f"{i+1}. {comment[:150]}" + "..." if len(comment)>150 else comment)
        if len(data["comments"]) > 20:
            print(f"... and {len(data['comments']) - 20} more comments.")
    else:
        print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30)
    print(f"Check console logs and '{screenshot_dir}' folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post content...
Post content scraped (length: 627).

--- Starting scroll and comment extraction ---
--- Loop/Scroll attempt #1 ---
Scrolling down...
  Found 18 potential comment <p> tags.
    Added 2 new unique comments.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #2 ---
Scrolling down...
  Found 18 potential comment <p> tags.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #3 ---
Scrolling down...
  Found 18 potential comment <p> tags.
  Current total comments scraped: 2. Scroll height: 4924
Scroll height unchanged and no new comments for 2 strikes. Assuming all loaded or stuck.

--- Finished comment sc

NameError: name 'screenshot_dir' is not defined

In [27]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def scrape_post_and_all_comments(url, max_main_loops=15, scroll_pause_time=2.5):
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_all_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        short_wait = WebDriverWait(driver, 5) # Shorter wait for elements within loops

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.")
        except Exception as e: print(f"Error 'X' pop-up: {e}")

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))

        print("\n--- Starting scroll and comment extraction ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        no_new_actions_or_comments_strikes = 0

        for i in range(max_main_loops):
            print(f"--- Main Loop Iteration #{i+1} ---")
            action_taken_this_loop = False
            comments_found_before_interactions = len(unique_comment_texts_scraped)

            # 1. Click ALL visible "查看N条回复" (Expand Replies)
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            # Loop to click these as new ones might appear after expanding others
            while True:
                clicked_an_expand_button_this_pass = False
                try:
                    expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                    if not expand_buttons: # print("  No 'Expand Replies' links found this pass.");
                        break

                    # Filter only visible buttons before attempting to click
                    visible_expand_buttons = [b for b in expand_buttons if b.is_displayed()]
                    if not visible_expand_buttons: # print("  No *visible* 'Expand Replies' links.");
                        break

                    print(f"  Found {len(visible_expand_buttons)} visible 'Expand Replies' links to click.")
                    for button in visible_expand_buttons:
                        try:
                            # print("    Scrolling to 'Expand Replies' and clicking...")
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", button)
                            time.sleep(0.5) # Wait for scroll
                            # Use a short wait for clickability
                            button_to_click = short_wait.until(EC.element_to_be_clickable(button))
                            driver.execute_script("arguments[0].click();", button_to_click) # JS click
                            # button_to_click.click() # Regular click
                            print(f"    Clicked 'Expand Replies': {button.text[:20]}")
                            action_taken_this_loop = True
                            clicked_an_expand_button_this_pass = True
                            time.sleep(1.5) # Wait for replies to load
                            # After clicking, the DOM might change, so we might need to re-find buttons in the next 'while True' iteration
                        except StaleElementReferenceException: print("    Stale 'Expand Replies' link during click, will re-evaluate."); break # Break inner for to re-find
                        except ElementNotInteractableException: print("    'Expand Replies' link not interactable, might be covered or disabled.")
                        except TimeoutException: print("    Timeout waiting for 'Expand Replies' to be clickable.")
                        except Exception as e_expand: print(f"    Error clicking one 'Expand Replies': {e_expand}")
                    if not clicked_an_expand_button_this_pass: # No visible ones were successfully clicked
                        break
                except Exception as e_find_expand: print(f"  Error finding 'Expand Replies': {e_find_expand}"); break
                if not clicked_an_expand_button_this_pass: break # If loop completes without any clicks

            # 2. Scroll down
            print("  Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            # 3. Attempt to scrape comments
            comment_text_xpath = "//div[@class='comment__item__main']/p" # This XPath seems correct
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                if comment_p_tags:
                    # print(f"  Found {len(comment_p_tags)} potential comment <p> tags on this pass.")
                    new_comments_this_pass = 0
                    for p_tag in comment_p_tags:
                        try:
                            comment_text = p_tag.text.strip()
                            if comment_text and comment_text not in unique_comment_texts_scraped:
                                unique_comment_texts_scraped.add(comment_text)
                                new_comments_this_pass += 1
                        except StaleElementReferenceException: continue
                        except Exception: continue # Ignore errors for individual comment text retrieval
                    if new_comments_this_pass > 0:
                        print(f"    Added {new_comments_this_pass} new unique comments this pass.")
                        action_taken_this_loop = True # Finding new comments is an action
            except Exception as e_find_comments: print(f"  Error finding comment <p> tags: {e_find_comments}")

            # 4. Click "展开查看更多" (Load More Main Comments)
            load_more_comments_xpath = "//div[contains(@class,'more-comment') and (contains(., '展开查看更多') or contains(., '加载更多'))]"
            try:
                # Use short_wait for this button as it might appear/disappear
                load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多/加载更多' button. Clicking...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", load_more_button); time.sleep(0.3)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("    Clicked '展开查看更多/加载更多'.")
                action_taken_this_loop = True
                time.sleep(scroll_pause_time + 1) # Wait longer after this action
            except TimeoutException: print("  '展开查看更多/加载更多' button not found or not clickable this pass.")
            except Exception as e_load_more: print(f"  Error clicking '展开查看更多': {e_load_more}")

            # 5. Check for loop termination conditions
            current_height = driver.execute_script("return document.body.scrollHeight")
            print(f"  Loop {i+1} end. Total unique comments: {len(unique_comment_texts_scraped)}. Scroll height: {current_height}. Last height: {last_height}")

            if not action_taken_this_loop and len(unique_comment_texts_scraped) == comments_found_before_interactions:
                no_new_actions_or_comments_strikes += 1
                print(f"  No new actions or comments strike: {no_new_actions_or_comments_strikes}")
            else:
                no_new_actions_or_comments_strikes = 0 # Reset if something happened

            if no_new_actions_or_comments_strikes >= 2:
                print("No new comments found and no interaction buttons successfully clicked for 2 consecutive loops. Assuming completion.")
                break

            last_height = current_height
            if i == max_main_loops - 1: print("Reached max main loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"main_loop_end_{i+1}.png"))

        scraped_data["comments"] = list(unique_comment_texts_scraped)
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_all_comments(target_url, max_main_loops=10, scroll_pause_time=2.5)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(data["post_content"]) # Print full post content
    else:
        print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"]):
            print(f"{i+1}. {comment}") # Print full comment
    else:
        print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30)
    print(f"Check console logs and folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_post_all_comments' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post content...
Post content scraped (length: 706).

--- Starting scroll and comment extraction ---
--- Main Loop Iteration #1 ---
  Scrolling down...
    Added 18 new unique comments this pass.
  '展开查看更多/加载更多' button not found or not clickable this pass.
  Loop 1 end. Total unique comments: 18. Scroll height: 4924. Last height: 4924
--- Main Loop Iteration #2 ---
  Scrolling down...
  '展开查看更多/加载更多' button not found or not clickable this pass.
  Loop 2 end. Total unique comments: 18. Scroll height: 4924. Last height: 4924
  No new actions or comments strike: 1
--- Main Loop Iteration #3 ---
  Scrolling down...
  '展开查看更多/加载